In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

pd.set_option('display.max_columns', 30)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# 1. Comparação da origem dos dados: CSV vs. API

Nessa análise, vamos comparar os dados obtidos do Portal da Transparência via API REST e via download de arquivos CSV diretamente do site.   

Começaremos organizando os arquivos obtidos via API (veja o arquivo 'consilta-despesas-API.ipynb' para mais detalhes sobre como esses dados foram obtidos)

## 1.1 Organização dos dados obtidos via API

In [2]:
custom_date_parser = lambda x: datetime.strptime(x, "%d/%m/%Y")

despesas_api = pd.read_csv("despesas-2020-via-API.csv",
                           parse_dates=['data'], 
                           date_parser=custom_date_parser)

despesas_api_jan = despesas_api.loc[pd.DatetimeIndex(despesas_api['data']).month==1]
despesas_api_jan.tail(3)

,data,documento,documentoResumido,observacao,funcao,subfuncao,programa,acao,subtitulo,localizadorGasto,fase,especie,favorecido,codigoFavorecido,nomeFavorecido,ufFavorecido,valor,ug,uo,orgao,orgaoSuperior,categoria,grupo,elemento,modalidade,numeroProcesso,autor
273,2020-01-31,154040152572020GR800035,2020GR800035,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Pagamento,Original,030001 - TRIBUNAL DE CONTAS DA UNIAO,030001,TRIBUNAL DE CONTAS DA UNIAO,DF,"1.536,25",FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,FUNDACAO UNIVERSIDADE DE BRASILIA,Fundação Universidade de Brasília,Ministério da Educação,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.007878/2020,Sem Informação
274,2020-01-31,154040152572020GR800034,2020GR800034,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Pagamento,Original,257001 - DIRETORIA EXECUTIVA DO FUNDO NAC. DE ...,257001,DIRETORIA EXECUTIVA DO FUNDO NAC. DE SAUDE,DF,"1.536,25",FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,FUNDACAO UNIVERSIDADE DE BRASILIA,Fundação Universidade de Brasília,Ministério da Educação,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.007878/2020,Sem Informação
275,2020-01-31,154040152572020GR800033,2020GR800033,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Pagamento,Original,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,154040,FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,DF,"508.065,00",FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,FUNDACAO UNIVERSIDADE DE BRASILIA,Fundação Universidade de Brasília,Ministério da Educação,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.007878/2020,Sem Informação


Na etapa de limpeza, vamos excluir as colunas que têm valores constantes e formatar a coluna de 'valor' em números tipo float.

In [3]:
def formatar_origem_api(input_df):
    df = input_df.copy()

    # Identificando e excluindo colunas de valor constante:
    colunas_constantes = df.columns[df.nunique() <= 1]
    print('As colunas constantes, que estão sendo eliminadas do dataframe, são: '
           + str(list(colunas_constantes)))
    df.drop(list(colunas_constantes), inplace=True, axis=1)

    # Formatando a coluna de valor:
    df['valor'] = [x.replace('.', '') for x in df['valor']]
    df['valor'] = [x.replace(' ', '') for x in df['valor']]
    df['valor'] = [x.replace(',', '.') for x in df['valor']]
    df['valor'] = df['valor'].astype(float)
    df.valor = df.valor.round(2)

    return df

In [4]:
despesas_api_jan = formatar_origem_api(despesas_api_jan)
despesas_api_jan.tail(3)

As colunas constantes, que estão sendo eliminadas do dataframe, são: ['fase', 'ug', 'orgao', 'orgaoSuperior', 'autor']


,data,documento,documentoResumido,observacao,funcao,subfuncao,programa,acao,subtitulo,localizadorGasto,especie,favorecido,codigoFavorecido,nomeFavorecido,ufFavorecido,valor,uo,categoria,grupo,elemento,modalidade,numeroProcesso
273,2020-01-31,154040152572020GR800035,2020GR800035,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Original,030001 - TRIBUNAL DE CONTAS DA UNIAO,030001,TRIBUNAL DE CONTAS DA UNIAO,DF,1536.25,FUNDACAO UNIVERSIDADE DE BRASILIA,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.007878/2020
274,2020-01-31,154040152572020GR800034,2020GR800034,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Original,257001 - DIRETORIA EXECUTIVA DO FUNDO NAC. DE ...,257001,DIRETORIA EXECUTIVA DO FUNDO NAC. DE SAUDE,DF,1536.25,FUNDACAO UNIVERSIDADE DE BRASILIA,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.007878/2020
275,2020-01-31,154040152572020GR800033,2020GR800033,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Original,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,154040,FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,DF,508065.00,FUNDACAO UNIVERSIDADE DE BRASILIA,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.007878/2020


Com isso, temos um dataframe com as despesas de Janeiro/2020, com base nos dados obtidos via API. Agora vamos organizar os dados obtidos via download de CSV.

## 1.2 Organização dos dados obtidos via download de CSV

In [5]:
despesas_csv_jan = pd.read_csv("despesas-2020-via-csvdownload/202001.csv", sep=';')
despesas_csv_jan.head()

,Mês Ano,Órgão Superior,Órgão/Entidade Vinculada,Unidade Gestora,Área de atuação (Função),Subfunção,Programa Orçamentário,Ação Orçamentária,Programa de Governo,Autor Emenda,Plano Orçamentário,Grupo de Despesa,Elemento de Despesa,Modalidade de Despesa,Valor Empenhado,Valor Liquidado,Valor Pago,Valor Restos a Pagar Pagos,Unnamed: 18
0,01/2020,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,10 - Saúde,122 - Administração geral,2015 - FORTALECIMENTO DO SISTEMA UNICO DE SAUD...,8287 - APRIMORAMENTO DA ARTICULACAO E COOPERAC...,00 - NAO ATRIBUIDO,2826 - ERIKA KOKAY,EIND - EMENDA INDIVIDUAL,3 - Outras Despesas Correntes,20 - Auxílio Financeiro a Pesquisadores,90 - Reserva de Contingência,"0,00","0,00","0,00","22.000,00",nan
1,01/2020,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,331 - Proteção e benefícios ao trabalhador,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,212B - BENEFICIOS OBRIGATORIOS AOS SERVIDORES ...,00 - NAO ATRIBUIDO,0000,0005 - AUXILIO-ALIMENTACAO DE CIVIS,3 - Outras Despesas Correntes,04 - Contratação por Tempo Determinado,90 - Reserva de Contingência,"500.000,00","106.089,45","0,00","0,00",nan
2,01/2020,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,11 - Trabalho,125 - Normatização e fiscalização,2071 - PROMOCAO DO TRABALHO DECENTE E ECONOMIA...,20YU - FISCALIZACAO DE OBRIGACOES TRABALHISTAS...,00 - NAO ATRIBUIDO,0000,0009 - FISCALIZACAO DE OBRIGACOES TRABALHISTAS...,3 - Outras Despesas Correntes,36 - Outros Serviços de Terceiros - Pessoa Física,90 - Reserva de Contingência,"0,00","0,00","0,00","10.645,80",nan
3,01/2020,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,2080 - EDUCACAO DE QUALIDADE PARA TODOS,8282 - REESTRUTURACAO E MODERNIZACAO DAS INSTI...,00 - NAO ATRIBUIDO,0000,0000 - REESTRUTURACAO E EXPANSAO DE INSTITUICO...,4 - Investimentos,52 - Equipamentos e Material Permanente,90 - Reserva de Contingência,"0,00","0,00","0,00","22.088,18",nan
4,01/2020,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,331 - Proteção e benefícios ao trabalhador,2109 - PROGRAMA DE GESTAO E MANUTENCAO DO MINI...,212B - BENEFICIOS OBRIGATORIOS AOS SERVIDORES ...,00 - NAO ATRIBUIDO,0000,0005 - AUXILIO-ALIMENTACAO DE CIVIS,3 - Outras Despesas Correntes,04 - Contratação por Tempo Determinado,90 - Reserva de Contingência,"0,00","0,00","0,00","118.865,54",nan


Na etapa de limpeza, renomear as colunas e formatar aas que se referem a valores em números tipo float.

In [6]:
def formatar_origem_csv(df):
    df_copy = df.rename(columns={'Mês Ano':'mes_ano',
                                 'Órgão Superior':'orgao_superior',
                                 'Órgão/Entidade Vinculada':'orgao_ent_vinculada',
                                 'Unidade Gestora':'unid_gestora',
                                 'Área de atuação (Função)':'area_atuacao',
                                 'Subfunção':'subfuncao',
                                 'Programa Orçamentário':'programa_orcamentario',
                                 'Ação Orçamentária':'acao_orcamentaria',
                                 'Programa de Governo':'programa_de_governo',
                                 'Autor Emenda':'autor_emenda',
                                 'Plano Orçamentário':'plano_orcamentario',
                                 'Grupo de Despesa':'grupo_de_despesa',
                                 'Elemento de Despesa':'elemento_de_despesa',
                                 'Modalidade de Despesa':'modalidade_de_despesa',
                                 'Valor Empenhado':'valor_empenhado',
                                 'Valor Liquidado':'valor_liquidado',
                                 'Valor Pago':'valor_pago',
                                 'Valor Restos a Pagar Pagos':'restos_a_pagar_pagos'})
    
    df_copy = df_copy.drop(['Unnamed: 18'], axis=1)
    
    df_copy['valor_empenhado'] = [x.replace('.', '') for x in df_copy['valor_empenhado']]
    df_copy['valor_empenhado'] = [x.replace(' ', '') for x in df_copy['valor_empenhado']]
    df_copy['valor_empenhado'] = [x.replace(',', '.') for x in df_copy['valor_empenhado']]
    df_copy['valor_empenhado'] = df_copy['valor_empenhado'].astype(float)
    
    df_copy['valor_liquidado'] = [x.replace('.', '') for x in df_copy['valor_liquidado']]
    df_copy['valor_liquidado'] = [x.replace(' ', '') for x in df_copy['valor_liquidado']]
    df_copy['valor_liquidado'] = [x.replace(',', '.') for x in df_copy['valor_liquidado']]
    df_copy['valor_liquidado'] = df_copy['valor_liquidado'].astype(float)
    
    df_copy['valor_pago'] = [x.replace('.', '') for x in df_copy['valor_pago']]
    df_copy['valor_pago'] = [x.replace(' ', '') for x in df_copy['valor_pago']]
    df_copy['valor_pago'] = [x.replace(',', '.') for x in df_copy['valor_pago']]
    df_copy['valor_pago'] = df_copy['valor_pago'].astype(float)
        
    df_copy['restos_a_pagar_pagos'] = [x.replace('.', '') for x in df_copy['restos_a_pagar_pagos']]
    df_copy['restos_a_pagar_pagos'] = [x.replace(' ', '') for x in df_copy['restos_a_pagar_pagos']]
    df_copy['restos_a_pagar_pagos'] = [x.replace(',', '.') for x in df_copy['restos_a_pagar_pagos']]
    df_copy['restos_a_pagar_pagos'] = df_copy['restos_a_pagar_pagos'].astype(float)
    
    df_copy['mes_ano'] = pd.to_datetime(df_copy['mes_ano'], format='%m/%Y')
    
    return df_copy

In [7]:
despesas_csv_jan = formatar_origem_csv(despesas_csv_jan)
despesas_csv_jan.head()

,mes_ano,orgao_superior,orgao_ent_vinculada,unid_gestora,area_atuacao,subfuncao,programa_orcamentario,acao_orcamentaria,programa_de_governo,autor_emenda,plano_orcamentario,grupo_de_despesa,elemento_de_despesa,modalidade_de_despesa,valor_empenhado,valor_liquidado,valor_pago,restos_a_pagar_pagos
0,2020-01-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,10 - Saúde,122 - Administração geral,2015 - FORTALECIMENTO DO SISTEMA UNICO DE SAUD...,8287 - APRIMORAMENTO DA ARTICULACAO E COOPERAC...,00 - NAO ATRIBUIDO,2826 - ERIKA KOKAY,EIND - EMENDA INDIVIDUAL,3 - Outras Despesas Correntes,20 - Auxílio Financeiro a Pesquisadores,90 - Reserva de Contingência,0.00,0.00,0.00,22000.00
1,2020-01-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,331 - Proteção e benefícios ao trabalhador,0032 - PROGRAMA DE GESTAO E MANUTENCAO DO PODE...,212B - BENEFICIOS OBRIGATORIOS AOS SERVIDORES ...,00 - NAO ATRIBUIDO,0000,0005 - AUXILIO-ALIMENTACAO DE CIVIS,3 - Outras Despesas Correntes,04 - Contratação por Tempo Determinado,90 - Reserva de Contingência,500000.00,106089.45,0.00,0.00
2,2020-01-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,11 - Trabalho,125 - Normatização e fiscalização,2071 - PROMOCAO DO TRABALHO DECENTE E ECONOMIA...,20YU - FISCALIZACAO DE OBRIGACOES TRABALHISTAS...,00 - NAO ATRIBUIDO,0000,0009 - FISCALIZACAO DE OBRIGACOES TRABALHISTAS...,3 - Outras Despesas Correntes,36 - Outros Serviços de Terceiros - Pessoa Física,90 - Reserva de Contingência,0.00,0.00,0.00,10645.80
3,2020-01-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,364 - Ensino superior,2080 - EDUCACAO DE QUALIDADE PARA TODOS,8282 - REESTRUTURACAO E MODERNIZACAO DAS INSTI...,00 - NAO ATRIBUIDO,0000,0000 - REESTRUTURACAO E EXPANSAO DE INSTITUICO...,4 - Investimentos,52 - Equipamentos e Material Permanente,90 - Reserva de Contingência,0.00,0.00,0.00,22088.18
4,2020-01-01,26000 - Ministério da Educação,26271 - Fundação Universidade de Brasília,154040 - FUNDACAO UNIVERSIDADE DE BRASILIA - FUB,12 - Educação,331 - Proteção e benefícios ao trabalhador,2109 - PROGRAMA DE GESTAO E MANUTENCAO DO MINI...,212B - BENEFICIOS OBRIGATORIOS AOS SERVIDORES ...,00 - NAO ATRIBUIDO,0000,0005 - AUXILIO-ALIMENTACAO DE CIVIS,3 - Outras Despesas Correntes,04 - Contratação por Tempo Determinado,90 - Reserva de Contingência,0.00,0.00,0.00,118865.54


## 1.3 Comparação dos valores obtidos: por ação

In [8]:
grupo_api_por_acao = despesas_api_jan.groupby(['acao'])['valor'].sum().reset_index()
grupo_api_por_acao

,acao,valor
0,-14 - Múltiplo,40343720.98
1,0487 - CONCESSAO DE BOLSAS DE ESTUDO NO ENSINO...,148696.87
2,"20GK - FOMENTO AS ACOES DE GRADUACAO, POS-GRAD...",4000.00
3,20RJ - APOIO A CAPACITACAO E FORMACAO INICIAL ...,159.00
4,20RK - FUNCIONAMENTO DE INSTITUICOES FEDERAIS ...,5084496.70
5,20TP - ATIVOS CIVIS DA UNIAO,49523895.51
6,"2250 - SELECAO, FORMACAO E DESENVOLVIMENTO DE ...",2469.43
7,4002 - ASSISTENCIA AO ESTUDANTE DE ENSINO SUPE...,7875.00
8,4572 - CAPACITACAO DE SERVIDORES PUBLICOS FEDE...,7562.13
9,"6702 - APOIO A PROJETOS E EVENTOS DE EDUCACAO,...",5311.30


In [19]:
grupo_csv_por_acao = despesas_csv_jan.groupby(['acao_orcamentaria'])['valor_pago'].sum().reset_index()
grupo_csv_por_acao

,acao_orcamentaria,valor_pago
0,00OQ - CONTRIBUICOES A ORGANISMOS INTERNACIONA...,0.00
1,00PW - CONTRIBUICOES A ENTIDADES NACIONAIS SEM...,0.00
2,0181 - APOSENTADORIAS E PENSOES CIVIS DA UNIAO,0.00
3,0487 - CONCESSAO DE BOLSAS DE ESTUDO NO ENSINO...,0.00
4,0536 - BENEFICIOS E PENSOES INDENIZATORIAS DEC...,0.00
5,"09HB - CONTRIBUICAO DA UNIAO, DE SUAS AUTARQUI...",12884025.81
6,156M - MODERNIZACAO ESTRATEGICA E OPERACIONAL ...,0.00
7,2000 - ADMINISTRACAO DA UNIDADE,0.00
8,2004 - ASSISTENCIA MEDICA E ODONTOLOGICA AOS S...,0.00
9,"20GK - FOMENTO AS ACOES DE GRADUACAO, POS-GRAD...",0.00


## 1.4 Comparação dos valores obtidos: por elemento

In [16]:
grupo_api_por_elemento = despesas_api_jan.groupby(['elemento'])['valor'].sum().reset_index()
grupo_api_por_elemento

,elemento,valor
0,-4 - Múltiplo,75304489.64
1,11 - Vencimentos e Vantagens Fixas - Pessoal C...,10264176.08
2,18 - Auxílio Financeiro a Estudantes,172593.63
3,20 - Auxílio Financeiro a Pesquisadores,167511.30
4,30 - Material de Consumo,56633.97
5,33 - Passagens e Despesas com Locomoção,61528.00
6,36 - Outros Serviços de Terceiros - Pessoa Física,12625.62
7,37 - Locação de Mão-de-Obra,2615091.96
8,39 - Outros Serviços de Terceiros - Pessoa Jur...,2853509.89
9,40 - Serviços de Tecnologia da Informação e Co...,3659.05


In [18]:
grupo_csv_por_elemento = despesas_csv_jan.groupby(['elemento_de_despesa'])['valor_pago'].sum().reset_index()
grupo_csv_por_elemento

,elemento_de_despesa,valor_pago
0,"01 - Aposentadorias do RPPS, Reserva Remunerad...",0.00
1,03 - Pensões do RPPS e do militar,0.00
2,04 - Contratação por Tempo Determinado,0.00
3,07 - Contribuição a Entidades Fechadas de Prev...,0.00
4,08 - Outros Benefícios Assistenciais do servid...,0.00
5,11 - Vencimentos e Vantagens Fixas - Pessoal C...,27419275.54
6,13 - Obrigações Patronais,12884105.26
7,14 - Diárias - Civil,0.00
8,16 - Outras Despesas Variáveis - Pessoal Civil,0.00
9,18 - Auxílio Financeiro a Estudantes,900.00


In [20]:
despesas_api_jan.head()

,data,documento,documentoResumido,observacao,funcao,subfuncao,programa,acao,subtitulo,localizadorGasto,especie,favorecido,codigoFavorecido,nomeFavorecido,ufFavorecido,valor,uo,categoria,grupo,elemento,modalidade,numeroProcesso
0,2020-01-02,154040152572020OB800038,2020OB800038,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,Original,00.000.000/0001-91 - BANCO DO BRASIL SA,00.000.000/0001-91,BANCO DO BRASIL SA,DF,20000000.00,FUNDACAO UNIVERSIDADE DE BRASILIA,M - Múltiplo,M - Múltiplo,-4 - Múltiplo,-4 - Múltiplo,23106.006576/2019-16
1,2020-01-02,154040152572020OB800037,2020OB800037,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,-14 - Múltiplo,Original,00.000.000/0001-91 - BANCO DO BRASIL SA,00.000.000/0001-91,BANCO DO BRASIL SA,DF,20000000.00,FUNDACAO UNIVERSIDADE DE BRASILIA,M - Múltiplo,M - Múltiplo,-4 - Múltiplo,-4 - Múltiplo,23106.006576/2019-16
2,2020-01-02,154040152572020OB800036,2020OB800036,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,2109 - PROGRAMA DE GESTAO E MANUTENCAO DO MINI...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Original,02.038.232/0001-64 - BANCO COOPERATIVO DO BRAS...,02.038.232/0001-64,BANCO COOPERATIVO DO BRASIL S/A,DF,1462417.50,FUNDACAO UNIVERSIDADE DE BRASILIA,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.006576/2019-16
3,2020-01-02,154040152572020OB800035,2020OB800035,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,2109 - PROGRAMA DE GESTAO E MANUTENCAO DO MINI...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Original,01.181.521/0001-55 - BANCO COOPERATIVO SICREDI...,01.181.521/0001-55,BANCO COOPERATIVO SICREDI S.A.,RS,26338.26,FUNDACAO UNIVERSIDADE DE BRASILIA,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.006576/2019-16
4,2020-01-02,154040152572020OB800034,2020OB800034,APROPRIACAO DA FOLHA DE PAGAMENTO DO SERVIDOR ...,12 - Educação,364 - Ensino superior,2109 - PROGRAMA DE GESTAO E MANUTENCAO DO MINI...,20TP - ATIVOS CIVIS DA UNIAO,20TP0053 - ATIVOS CIVIS DA UNIAO ...,0053 - NO DISTRITO FEDERAL,Original,60.701.190/0001-04 - ITAU UNIBANCO S.A.,60.701.190/0001-04,ITAU UNIBANCO S.A.,SP,3542903.35,FUNDACAO UNIVERSIDADE DE BRASILIA,3 - DESPESAS CORRENTES,1 - Pessoal e Encargos Sociais,11 - Vencimentos e Vantagens Fixas - Pessoal C...,90 - Reserva de Contingência,23106.006576/2019-16
